In [2]:
# for loading/processing the images  
from keras.preprocessing.image import load_img 
from keras.preprocessing.image import img_to_array 
from keras.applications.vgg16 import preprocess_input 

# models 
from keras.applications.vgg16 import VGG16 
from keras.models import Model

# clustering and dimension reduction
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# for everything else
import os
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import pandas as pd
import pickle

2024-06-29 13:12:14.635612: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-29 13:12:14.641239: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-29 13:12:14.654094: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-29 13:12:14.679548: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-29 13:12:14.679583: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-29 13:12:14.693834: I tensorflow/core/platform/cpu_feature_guard.cc:

In [3]:
path = './boxes/'
# os.system(f"aws s3 cp {'s3://olm-processed-s3/boxes/'} {path} --recursive")  

In [5]:
# this list holds all the image filename
boxes_file_paths = []

# reads paths for each image and stores it in the array
for path_tuple in os.walk(path):
    for file in path_tuple[2]:
        if file.endswith(".jpg"):
            boxes_file_paths.append(os.path.join(path_tuple[0], file))

In [13]:

img = load_img(boxes_file_paths[7], target_size=(224,224))
img = np.array(img)
reshaped_img = img.reshape(1,224,224,3)

(224, 224, 3)

In [14]:
# load model
model = VGG16()
# remove the output layer
model = Model(inputs=model.inputs, outputs=model.layers[-2].output)

2024-06-29 13:31:19.003233: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2024-06-29 13:31:19.372190: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2024-06-29 13:31:19.533292: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.


553467096/553467096 ━━━━━━━━━━━━━━━━━━━━ 41s 0us/step


2024-06-29 13:32:02.826518: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.


In [18]:
def extract_features(file, model):
    # load the image as a 224x224 array
    img = load_img(file, target_size=(224,224))
    # convert from 'PIL.Image.Image' to numpy array, then reshape for batch dimentions
    reshaped_img = np.array(img).reshape(1,224,224,3)  
    # prepare image for model
    imgx = preprocess_input(reshaped_img)
    # get the feature vector
    features = model.predict(imgx)
    return features

In [19]:
x = extract_features(boxes_file_paths[7], model)
x.shape

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 595ms/step


(1, 4096)